In [76]:
import numpy as np
import pandas as pd
import warnings
import random
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning) 

In [77]:
map_ = {'a':0 , 'c':1 , 'g':2 , 't':3}
W = None
ttl_a = None
ttl_c = None
ttl_g = None
ttl_t = None
total_bases = None
P0 = None
Z = None
Pk = None
seqs = None

In [78]:
def Clear_Global():
  global W 
  global ttl_a
  global ttl_c
  global ttl_g
  global ttl_t
  global total_bases
  global P0
  global Z
  global Pk
  global seqs
  
  W = None
  ttl_a = 0
  ttl_c = 0
  ttl_g = 0
  ttl_t = 0
  total_bases = 0
  P0 = None
  Z = []
  Pk = []
  seqs = None

In [79]:
def Read_File(path):
  global seqs
  global W
  given_motif_length = ''
  seqs_ = []
  f = open(path)
  for i , line in enumerate(f.readlines()):
    # print(line)
    if i == 0 :
      for j in reversed(range(len(line))):
        if str(line[j]).isnumeric():
          given_motif_length = str(line[j]) + given_motif_length
    elif line != '\n' and line.startswith(">s") == False:
      seq = ""
      for char in str(line[:-1]).lower():
        if char == 'a' or char == 't' or char == 'c' or char == 'g':
          seq += char
        else:
          print("invalid base : " , char)
      seqs_.append(seq)
  f.close()
  seqs = np.array(seqs_)
  W = int(given_motif_length)
  # print(W , len(seqs))

# for i in range(1 , 9):
#   Read_File('final_testing_data/Q' + str(i) +'.txt')

In [80]:
def Setup_Background():
  global ttl_a
  global ttl_c
  global ttl_g
  global ttl_t
  global total_bases
  global P0
  global seqs
  for seq in seqs:
    for base in seq:
      if base == 'a':ttl_a +=1
      if base == 'c':ttl_c +=1
      if base == 'g':ttl_g +=1
      if base == 't':ttl_t +=1
      if base != 'a' and base != 't' and base != 'c' and base != 'g' : print(base)
  total_bases = ttl_a + ttl_c + ttl_g + ttl_t
  P0 = np.array([ttl_a/total_bases , ttl_c/total_bases , ttl_g/total_bases , ttl_t/total_bases])

In [81]:
def random_initialize():
  global Pk
  global Z
  global seqs
  global W
  for i in range(4):
    r = np.random.random(W)
    r /= r.sum()
    Pk.append(r)
  Pk = np.array(Pk)
  for i in range(len(seqs)):
    r = np.random.random(len(seqs[i]) - W + 1) 
    r /= r.sum()
    Z.append(r)
  Z = np.array(Z)

In [82]:
def E_step():
  global seqs
  global P0
  global W
  global Z
  global Pk
  for i , seq in enumerate(seqs):  # modify each row in Z
    Zi = [] # Compute Zi , which means model the i th seq
    for j in range(len(seq) - W + 1): # enumerate all possible motif starting point
      cnt = 0
      Zij = 1
      for plc , base in enumerate(seq): # scan through i th seq
        # print(Zij)
        if plc < j or plc > j+W-1: # outside motif
          Zij *= P0[map_[base]] * pow(np.e , 1) # times e^1 to avoid Zij go down to smaller than e^-320 and then become 0
        else: # inside motif
          Zij *= Pk[map_[base]][cnt] * pow(np.e , 1)
          cnt+=1
      Zi.append(Zij) # finish compute Zij
    Zi = np.array(Zi) # normalized Zi
    Zi /= Zi.sum()
    for k in range(len(Zi)): # Rewrite Zi back to Z matrix
      Z[i][k] = Zi[k]

def modify_Pk(dc , db):
  global W
  global Z
  global seqs
  total_Zsum = 0
  for i in range(len(Z)):
    for j in range(len(Z[i])):
      total_Zsum += Z[i][j]
  for base in ['a' , 'c' , 'g' , 't']:
    for s in range(W):
      possible_place = []
      for seq in seqs:
        target = []
        for id , b in enumerate(seq):
          if b == base and (id - s) >= 0 and (W - s + id - 1) <= len(seq)-1 : 
            target.append(id)
        possible_place.append(target)
      Zsum = 0
      for i in range(len(possible_place)):
        if len(possible_place[i]) == 0: # i-th seq does not satisfy P (base , s)
          continue
        for j in possible_place[i]:
          Zsum += Z[i][j-s]
      Pk[map_[base]][s] = (Zsum+dc)*1.0/(total_Zsum+db)
    

def M_step(dc = 1 , db = 4): # psuedo counts for a base , and 4 bases
  modify_Pk(dc , db)

# def L_of_theta(): # stoppong criteria of the EM algorithm
#   # print(Pk)
#   L_of_theta = 0 + 1e-10
#   for i in range(len(seqs)):
#     sum_of_zi = 0 + 1e-10 #sigma of p(x^i , z^i , theta(i.e , p))
#     for j in range(len(Z[i])):
#       # Zij = 1
#       prob = 1
#       for plc , base in enumerate(seqs[i]):
#         if plc < j or plc > j + W - 1:
#           prob *= P0[map_[base]]
#         else:
#           prob *= Pk[map_[base]][plc - j]
#       sum_of_zi += prob
#     log_of_sum_of_zi = np.log(sum_of_zi)
#     L_of_theta += log_of_sum_of_zi
#   return L_of_theta


In [83]:
def find_motif():
  global seqs
  global Z
  global W
  ans = []
  ans_seq = []
  for i in range(len(seqs)):
    best_prob = -1
    best_pos = 0
    for j , prob in enumerate(Z[i]):
      if prob >= best_prob:
        best_prob = prob
        best_pos = j
    ans.append(best_pos)
    ans_seq.append(seqs[i][best_pos : best_pos + W])
  return np.array(ans) , np.array(ans_seq)

In [84]:
def hamming(a , b):
  cnt = 0 
  for i in range(len(a)):
    if a[i] != b[i]:
      cnt += 1
  return cnt

In [85]:
Qpaths = []
# Qpaths.append('project1_beta/Q1.txt')
# Qpaths.append('project1_beta/Q3.txt')
for i in range(1 , 9):
    Qpaths.append('final_testing_data/Q' + str(i) + '.txt')
# print(Qpaths)
vote_num = 5

['project1_beta/Q1.txt', 'project1_beta/Q3.txt', 'final_testing_data/Q1.txt', 'final_testing_data/Q2.txt', 'final_testing_data/Q3.txt', 'final_testing_data/Q4.txt', 'final_testing_data/Q5.txt', 'final_testing_data/Q6.txt', 'final_testing_data/Q7.txt', 'final_testing_data/Q8.txt']


In [86]:
for testcase_num , path in enumerate(Qpaths):
  voter_of_None = 0
  possible_final_output = []
  possible_final_plc = []
  possible_final_motifs = []
  for voter in range(vote_num):
    Clear_Global()
    Read_File(path)
    Setup_Background()
    random_initialize()
    it = 0
    cnt = 0
    while cnt <= 40 : 
      E_step()
      M_step()
      cnt +=1
    
    found_motif_start , found_motifs_for_each_seqs = find_motif()

    records = []
    for i in range(W):
      record = [0.0 , 0.0 , 0.0 , 0.0]
      for motif in found_motifs_for_each_seqs:
        record[map_[motif[i]]] += 1
      records.append(np.array(record))
    

    found_motif_by_this_voter = "" # 找出這個voter提出的motif長相
    for i in range(W):
      possible = []
      if max(records[i]) == records[i][0] : possible.append('a')
      if max(records[i]) == records[i][1] : possible.append('c')
      if max(records[i]) == records[i][2] : possible.append('g') 
      if max(records[i]) == records[i][3] : possible.append('t')
      choose = random.choice(possible)
      found_motif_by_this_voter += choose
    possible_final_output.append(found_motif_by_this_voter)
    possible_final_plc.append(found_motif_start)
    possible_final_motifs.append(found_motifs_for_each_seqs)
    
    n = 0 # 在這個voter的sequence中有幾個sequence不符合限制條件
    for motif in found_motifs_for_each_seqs:
      if hamming(found_motif_by_this_voter , motif) > 0.5 * W:
        n += 1
    if n >= 0.5 * len(found_motifs_for_each_seqs): # 如果有一半以上的sequence of this voter不符合限制條件 , 則這個voter標註為他提出"None"
      voter_of_None += 1

  if voter_of_None >= 0.5 * vote_num : # 如果有一半以上的voter提出"None" , 則最後的答案就是"None"
    print("Key = None")
  else :
    records_ = []
    for i in range(W):
      record = [0.0 , 0.0 , 0.0 , 0.0]
      for v in possible_final_output:
        record[map_[v[i]]] += 1
      records_.append(np.array(record))
    
    final_output = "" # 選出這些voters共同決定的最終output
    for i in range(W):
      possible = []
      if max(records_[i]) == records_[i][0] : possible.append('a')
      if max(records_[i]) == records_[i][1] : possible.append('c')
      if max(records_[i]) == records_[i][2] : possible.append('g') 
      if max(records_[i]) == records_[i][3] : possible.append('t')
      choose = random.choice(possible)
      final_output += choose

    best_hamming = W + 20
    choice_again = []
    for i , v in enumerate(possible_final_output): # 選擇一個voter , 他的found_motif和最終答案(final_output)有最近hamming distance , 
                                                   # 根據這個條件output這個voter相應的 motif starting place 結果做輸出
      if hamming(final_output , v) < best_hamming:
        choice_again = []
        choice_again.append(i)
        best_hamming = hamming(final_output , v)
      elif hamming(final_output , v) == best_hamming:
        choice_again.append(i)
    target = random.choice(choice_again)
    best_plc = possible_final_plc[target]
    best_motifs = possible_final_motifs[target]
    path = 'Ans' + str(testcase_num + 1) +'.txt'
    f = open(path, 'w')
    print("Key = " , final_output.upper() , file = f)
    print("Key = " , final_output.upper())
    for i , plc in enumerate(best_plc):
      print(f"position {plc} on seq#{i+1}" )
      print(best_motifs[i].upper())
    print("==============================")
    f.close()

Key =  CGCGCGCGCG
Key =  GGGCCGGGATTAGGG
Key =  AATTATTTAAAAACC
Key =  TGTATATAAAAATTG
Key =  AACCCCCGGGGGCCC
Key =  CCAGGAGCCCCCCGG
Key =  TCAACTCTATTTCTC
Key =  CCTCTCTAACTCTAC
Key =  CCCCCCCCCCCCCCC
Key =  CCCCCCCCCCCCCCC
